# In this file, graph databases are created in neo4j using the data from the created CSV files about groups, events and RSVPs.

# 1. preparation: required libraries are fetched and connection to neo4j is established.

In [7]:
import os
import csv
from neo4j import GraphDatabase, basic_auth

#currently neo4j is running on localhost
neo4jUrl = os.environ.get('NEO4J_URL',"bolt://192.168.99.100")
neo4jUser = os.environ.get('NEO4J_USER',"neo4j")
neo4jPass = os.environ.get('NEO4J_PASSWORD',"1")

driver = GraphDatabase.driver(neo4jUrl, auth=basic_auth(neo4jUser, neo4jPass))

graph = driver.session()
    
   

# 2. function for graph creation is prepared.
Create graph databases for groups, events, participants, topics, venues and the relationships between them using the specified csv files.

In [8]:
def createGraphUsingCSV(eventRange):

        fileGroups = "groups" + eventRange + ".csv"
        fileRSVPs = "rsvps" + eventRange + ".csv"
        fileEvents = "events" + eventRange + ".csv"
        
        with open(fileGroups, encoding='utf8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                id = row['id']
                name = row['name']
                members = row['members']
                link = row['link']
                topics = row['topics']

                graph.run(" with apoc.convert.fromJsonList({topics}) as list unwind list as list1 merge(t:Topic{id:list1.id, urlkey:list1.urlkey}) MERGE (g:Group {id: {id}, name: {name}, members: {members}, link:{link}, topics:{topics}}) with g, apoc.convert.fromJsonList(g.topics) as list2 unwind list2 as list3 match(tt:Topic) where tt.id in list3.id merge (tt)<-[:IS_ABOUT]-(g)",  
            
                            id = id, name = name, members = members, link = link, topics=topics)
   

        with open(fileEvents, encoding='utf8') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    group_id = row['group_id']
                    id = row['id']
                    rsvp_limit = row['rsvp_limit']
                    yes_rsvp_count = row['yes_rsvp_count']
                    city = row['city']
                    address = row['address']
                    venue_id = row['venue_id']
                    venue_name = row['venue_name']
                    lat = row['lat']
                    lon = row['lon']
                    country = row['country']
                    name = row['name']
                    event_url = row['event_url']
                    time = row['time']
                    graph.run ("Merge (v:Venue {id:{venue_id}, venue_name:{venue_name}, lat:{lat}, lon:{lon}, address:{address}, city:{city}}) with v MATCH (c:Group) where toString(c.id) = toString({group_id}) MERGE (e:Event {id: {id}, name: {name}, url : {event_url}, rsvp_limit : {rsvp_limit}, yes_rsvp_count:{yes_rsvp_count}, time : {time}, venue_id:{venue_id}}) MERGE (e)<-[:ORGANIZES]-(c) with e Match (v2:Venue) where v2.id = e.venue_id MERGE (v2)<-[:TAKES_PLACE_AT]-(e)"
                              , venue_id = venue_id, venue_name = venue_name, lat = lat, lon = lon, address = address, group_id = group_id, id = id, name = name, event_url = event_url, rsvp_limit = rsvp_limit, yes_rsvp_count = yes_rsvp_count, time = time, city = city
                              )


        with open(fileRSVPs, encoding='utf8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                event_id = row['event_id']
                member_id = row['member_id']
                name = row['name']
                graph.run("MATCH (e:Event) where toString(e.id) = toString({event_id}) MERGE (p:Person {id: {member_id}, name: {name}}) MERGE (e)<-[:RSVPd]-(p)", event_id = event_id, member_id = member_id, name = name)           
     

       

# 3. the function for graph creation is called here. The filenames are passed as parameters.
Depending on the desired analysis, the corresponding line is used. The other lines are commented out. 

In [9]:
#createGraphUsingCSV("ZH")
createGraphUsingCSV("Oracle")

#createGraphUsingCSV("Top")
createGraphUsingCSV("ZHTrend")

graph.close()